In [2]:
import os

In [1]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [5]:
import pyodbc

try:
    conn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};"
                          "SERVER=LAPTOP-RES6G37K;"
                          "DATABASE=adventureworks2019;"
                          "Trusted_connection=Yes;")
    print("✅ SQL Server Login Successful!")
    conn.close()
except Exception as e:
    print("❌ SQL Server Login Failed:", e)

✅ SQL Server Login Successful!


In [6]:
from sqlalchemy import create_engine

In [23]:
from sqlalchemy.engine import URL
server = "LAPTOP-RES6G37K"
database = "adventureworks2019"


connection_url = URL.create(
    "mssql+pyodbc",
    host="tcp:" + server,
    port=1433,
    database=database,
    
    # driver is typically required in the query
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "Trusted_Connection": "yes"
    },
)

engine = create_engine(connection_url)


In [24]:
try:
    with engine.connect() as conn:
        print("✅ SQLAlchemy connected successfully!")
except Exception as e:
    print("❌ SQLAlchemy connection failed:", e)

✅ SQLAlchemy connected successfully!


In [46]:
from langchain.utilities import SQLDatabase

In [100]:
from sqlalchemy import create_engine, MetaData

meta = MetaData()
# Reflect *all* schemas and tables:
for schema_name in ["Person"]:
    meta.reflect(bind=engine, schema=schema_name)



C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\1818128909.py:6: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  meta.reflect(bind=engine, schema=schema_name)


In [101]:
meta.create_all(bind=engine)

In [102]:
# Now pass this pre-reflected MetaData to LangChain:
dbase = SQLDatabase(engine=engine, metadata=meta)

In [103]:
dbase.get_table_names()

['AWBuildVersion', 'DatabaseLog', 'ErrorLog']

In [104]:
from langchain.sql_database import SQLDatabase

all_tables = list(meta.tables.keys())
# e.g. ["dbo.SomeTable", "MyOtherSchema.OtherTable", ...]



In [105]:
len(all_tables)

17

In [107]:
db = SQLDatabase(
    engine=engine,
    metadata=meta,
    include_tables=all_tables)


ValueError: include_tables {'Person.AddressType', 'Person.BusinessEntityContact', 'Sales.SalesTerritory', 'Person.ContactType', 'Person.PhoneNumberType', 'Person.BusinessEntity', 'Person.Address', 'Person.Person', 'Person.PersonPhone', 'Person.BusinessEntityAddress', 'Person.CountryRegion', 'Person.StateProvince', 'Person.EmailAddress', 'Person.Password'} not found in database

In [71]:
for table_name, table_obj in metadata.tables.items():
    print(table_name, "=>", table_obj)

Person.Address => Person.Address
Person.StateProvince => Person.StateProvince
Person.CountryRegion => Person.CountryRegion
Sales.SalesTerritory => Sales.SalesTerritory
Person.AddressType => Person.AddressType
Person.BusinessEntity => Person.BusinessEntity
Person.BusinessEntityAddress => Person.BusinessEntityAddress
Person.BusinessEntityContact => Person.BusinessEntityContact
Person.ContactType => Person.ContactType
Person.Person => Person.Person
Person.EmailAddress => Person.EmailAddress
Person.Password => Person.Password
Person.PersonPhone => Person.PersonPhone
Person.PhoneNumberType => Person.PhoneNumberType
HumanResources.Department => HumanResources.Department
HumanResources.Employee => HumanResources.Employee
HumanResources.EmployeeDepartmentHistory => HumanResources.EmployeeDepartmentHistory
HumanResources.Shift => HumanResources.Shift
HumanResources.EmployeePayHistory => HumanResources.EmployeePayHistory
HumanResources.JobCandidate => HumanResources.JobCandidate
Production.BillO

In [84]:
dbase.get_table_names()

['AWBuildVersion', 'DatabaseLog', 'ErrorLog']

In [16]:
from langchain_openai import ChatOpenAI

In [17]:
import os
from langchain_openai import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.utilities import SQLDatabase
from sqlalchemy import create_engine

In [18]:
# Create SQLDatabase instance
db = SQLDatabase(engine)

In [ ]:
print(db)

{'ErrorLog', 'DatabaseLog', 'AWBuildVersion'}


In [114]:
from sqlalchemy import create_engine, MetaData,URL
from langchain.sql_database import SQLDatabase
server = "LAPTOP-RES6G37K"
database = "adventureworks2019"
connection_url = URL.create(
    "mssql+pyodbc",
    host="tcp:" + server,
    port=1433,
    database=database,
    
    # driver is typically required in the query
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "Trusted_Connection": "yes"
    },
)
engine = create_engine(connection_url)
# Create a fresh MetaData object
metadata = MetaData()

# Reflect the default (dbo) schema
metadata.reflect(bind=engine, schema="dbo")

# Reflect the other schemas you need
metadata.reflect(bind=engine, schema="Person")


# Now metadata.tables should include tables from all reflected schemas
print("Reflected tables:", metadata.tables.keys())
# e.g. dict_keys(['dbo.Customers', 'MyOtherSchema.Orders', 'Sales.Invoices', ...])




C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\2538543894.py:25: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  metadata.reflect(bind=engine, schema="Person")


Reflected tables: dict_keys(['dbo.AWBuildVersion', 'dbo.DatabaseLog', 'dbo.ErrorLog', 'Person.Address', 'Person.StateProvince', 'Person.CountryRegion', 'Sales.SalesTerritory', 'Person.AddressType', 'Person.BusinessEntity', 'Person.BusinessEntityAddress', 'Person.BusinessEntityContact', 'Person.ContactType', 'Person.Person', 'Person.EmailAddress', 'Person.Password', 'Person.PersonPhone', 'Person.PhoneNumberType'])


In [110]:
from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()

orders_table = Table(
    "Orders",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("description", String),
    schema="Sales"   # <--- specify the schema explicitly
)

metadata.create_all(bind=engine)

db = SQLDatabase(engine=engine, metadata=metadata)
print("Tables recognized by LangChain:", db.get_table_names())


Tables recognized by LangChain: ['AWBuildVersion', 'DatabaseLog', 'ErrorLog']


In [117]:
db = SQLDatabase(
    engine=engine,
    metadata=metadata,
    include_tables=["Person.BusinessEntity"]
)

ValueError: include_tables {'Person.BusinessEntity'} not found in database

In [115]:
metadata.tables.keys()

dict_keys(['dbo.AWBuildVersion', 'dbo.DatabaseLog', 'dbo.ErrorLog', 'Person.Address', 'Person.StateProvince', 'Person.CountryRegion', 'Sales.SalesTerritory', 'Person.AddressType', 'Person.BusinessEntity', 'Person.BusinessEntityAddress', 'Person.BusinessEntityContact', 'Person.ContactType', 'Person.Person', 'Person.EmailAddress', 'Person.Password', 'Person.PersonPhone', 'Person.PhoneNumberType'])

In [118]:
print("All tables reflected in metadata:")
for t in metadata.tables:
    print(repr(t))

All tables reflected in metadata:
'dbo.AWBuildVersion'
'dbo.DatabaseLog'
'dbo.ErrorLog'
'Person.Address'
'Person.StateProvince'
'Person.CountryRegion'
'Sales.SalesTerritory'
'Person.AddressType'
'Person.BusinessEntity'
'Person.BusinessEntityAddress'
'Person.BusinessEntityContact'
'Person.ContactType'
'Person.Person'
'Person.EmailAddress'
'Person.Password'
'Person.PersonPhone'
'Person.PhoneNumberType'


In [119]:
print("Tables in metadata after reflection:", list(metadata.tables.keys()))

Tables in metadata after reflection: ['dbo.AWBuildVersion', 'dbo.DatabaseLog', 'dbo.ErrorLog', 'Person.Address', 'Person.StateProvince', 'Person.CountryRegion', 'Sales.SalesTerritory', 'Person.AddressType', 'Person.BusinessEntity', 'Person.BusinessEntityAddress', 'Person.BusinessEntityContact', 'Person.ContactType', 'Person.Person', 'Person.EmailAddress', 'Person.Password', 'Person.PersonPhone', 'Person.PhoneNumberType']


In [121]:
from langchain.sql_database import SQLDatabase

all_tables = list(metadata.tables.keys())  # Should now include 'Person.BusinessEntity'

db = SQLDatabase(
    engine=engine,
    metadata=metadata,
    include_tables=list(metadata.tables.keys())  # Must match exactly
)

ValueError: include_tables {'Person.AddressType', 'Person.BusinessEntityContact', 'Sales.SalesTerritory', 'dbo.AWBuildVersion', 'Person.ContactType', 'Person.PhoneNumberType', 'Person.BusinessEntity', 'Person.Address', 'Person.Person', 'Person.PersonPhone', 'Person.BusinessEntityAddress', 'dbo.DatabaseLog', 'Person.CountryRegion', 'Person.StateProvince', 'Person.EmailAddress', 'dbo.ErrorLog', 'Person.Password'} not found in database

In [122]:
# Create metadata
metadata = MetaData()

# Reflect each schema you need
for schema_name in ["dbo", "Person", "Sales"]:
    metadata.reflect(bind=engine, schema=schema_name)

# Inspect what was reflected
print("SQLAlchemy sees tables:", list(metadata.tables.keys()))

C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\1546482088.py:6: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  metadata.reflect(bind=engine, schema=schema_name)
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\1546482088.py:6: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  metadata.reflect(bind=engine, schema=schema_name)


SQLAlchemy sees tables: ['dbo.AWBuildVersion', 'dbo.DatabaseLog', 'dbo.ErrorLog', 'Person.Address', 'Person.StateProvince', 'Person.CountryRegion', 'Sales.SalesTerritory', 'Person.AddressType', 'Person.BusinessEntity', 'Person.BusinessEntityAddress', 'Person.BusinessEntityContact', 'Person.ContactType', 'Person.Person', 'Person.EmailAddress', 'Person.Password', 'Person.PersonPhone', 'Person.PhoneNumberType', 'Sales.CountryRegionCurrency', 'Sales.Currency', 'Sales.CreditCard', 'Sales.CurrencyRate', 'Sales.Customer', 'Sales.Store', 'Sales.SalesPerson', 'HumanResources.Employee', 'Sales.PersonCreditCard', 'Sales.SalesOrderDetail', 'Sales.SalesOrderHeader', 'Purchasing.ShipMethod', 'Sales.SpecialOfferProduct', 'Production.Product', 'Production.ProductModel', 'Production.ProductSubcategory', 'Production.ProductCategory', 'Production.UnitMeasure', 'Sales.SpecialOffer', 'Sales.SalesOrderHeaderSalesReason', 'Sales.SalesReason', 'Sales.SalesPersonQuotaHistory', 'Sales.SalesTaxRate', 'Sales.Sale

In [123]:
from langchain.sql_database import SQLDatabase

db = SQLDatabase(
    engine=engine,
    metadata=metadata  # You are bypassing LangChain's internal re-reflection
)

# Confirm
print("LangChain sees:", db.get_table_names())


LangChain sees: ['AWBuildVersion', 'DatabaseLog', 'ErrorLog']


In [124]:
print("LangChain _all_tables:", db._all_tables)

LangChain _all_tables: {'ErrorLog', 'DatabaseLog', 'AWBuildVersion'}


In [125]:
print("Metadata tables:", metadata.tables.keys())

Metadata tables: dict_keys(['dbo.AWBuildVersion', 'dbo.DatabaseLog', 'dbo.ErrorLog', 'Person.Address', 'Person.StateProvince', 'Person.CountryRegion', 'Sales.SalesTerritory', 'Person.AddressType', 'Person.BusinessEntity', 'Person.BusinessEntityAddress', 'Person.BusinessEntityContact', 'Person.ContactType', 'Person.Person', 'Person.EmailAddress', 'Person.Password', 'Person.PersonPhone', 'Person.PhoneNumberType', 'Sales.CountryRegionCurrency', 'Sales.Currency', 'Sales.CreditCard', 'Sales.CurrencyRate', 'Sales.Customer', 'Sales.Store', 'Sales.SalesPerson', 'HumanResources.Employee', 'Sales.PersonCreditCard', 'Sales.SalesOrderDetail', 'Sales.SalesOrderHeader', 'Purchasing.ShipMethod', 'Sales.SpecialOfferProduct', 'Production.Product', 'Production.ProductModel', 'Production.ProductSubcategory', 'Production.ProductCategory', 'Production.UnitMeasure', 'Sales.SpecialOffer', 'Sales.SalesOrderHeaderSalesReason', 'Sales.SalesReason', 'Sales.SalesPersonQuotaHistory', 'Sales.SalesTaxRate', 'Sales.S

In [126]:
with engine.connect() as conn:
    result = conn.execute("SELECT TOP 5 * FROM Person.BusinessEntity")
    for row in result:
        print(row)

ObjectNotExecutableError: Not an executable object: 'SELECT TOP 5 * FROM Person.BusinessEntity'

In [127]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SELECT TOP 5 * FROM Person.BusinessEntity"))
    for row in result:
        print(row)

(1, '0C7D8F81-D7B1-4CF0-9C0A-4CD8B6B50087', datetime.datetime(2017, 12, 13, 13, 20, 24, 150000))
(2, '6648747F-7843-4002-B317-65389684C398', datetime.datetime(2017, 12, 13, 13, 20, 24, 430000))
(3, '568204DA-93D7-42F4-8A7A-4446A144277D', datetime.datetime(2017, 12, 13, 13, 20, 24, 540000))
(4, '0EFF57B9-4F4F-41A6-8867-658C199A5FC0', datetime.datetime(2017, 12, 13, 13, 20, 24, 570000))
(5, 'B82F88D1-FF79-4FD9-8C54-9D24C140F647', datetime.datetime(2017, 12, 13, 13, 20, 24, 633000))


In [128]:
from langchain.sql_database import SQLDatabase

class PatchedSQLDatabase(SQLDatabase):
    def _validate_table_names(self):
        # Disable LangChain's strict table filtering logic
        return

db = PatchedSQLDatabase(engine=engine, metadata=metadata)
print("Patched LangChain sees:", db.get_table_names())


Patched LangChain sees: ['AWBuildVersion', 'DatabaseLog', 'ErrorLog']


In [129]:
non_dbo_tables = [name for name in metadata.tables.keys() if not name.startswith("dbo.")]
db = SQLDatabase(engine=engine, metadata=metadata, include_tables=non_dbo_tables)


ValueError: include_tables {'Person.AddressType', 'Sales.SpecialOfferProduct', 'Person.ContactType', 'Sales.CreditCard', 'Sales.SalesPerson', 'Sales.SalesTerritoryHistory', 'Person.Address', 'Sales.SalesOrderHeaderSalesReason', 'Person.Person', 'Person.PersonPhone', 'Person.BusinessEntityAddress', 'Sales.CurrencyRate', 'Sales.SalesOrderDetail', 'Person.CountryRegion', 'Person.StateProvince', 'HumanResources.Employee', 'Person.EmailAddress', 'Sales.SalesOrderHeader', 'Production.UnitMeasure', 'Person.Password', 'Sales.ShoppingCartItem', 'Production.Product', 'Production.ProductSubcategory', 'Sales.Customer', 'Sales.Store', 'Sales.PersonCreditCard', 'Production.ProductCategory', 'Sales.SalesTerritory', 'Sales.SpecialOffer', 'Sales.SalesPersonQuotaHistory', 'Person.PhoneNumberType', 'Person.BusinessEntity', 'Sales.Currency', 'Sales.SalesTaxRate', 'Purchasing.ShipMethod', 'Sales.CountryRegionCurrency', 'Sales.SalesReason', 'Production.ProductModel', 'Person.BusinessEntityContact'} not foun

In [130]:
from langchain.sql_database import SQLDatabase

class MyCustomSQLDatabase(SQLDatabase):
    def _get_all_tables(self):
        # Instead of using the engine inspector, we use our metadata's table names.
        # This bypasses the default behavior that only looks at dbo.
        return set(self.metadata.tables.keys())

# Then initialize:
db = MyCustomSQLDatabase(engine=engine, metadata=metadata)
print("Custom DB tables:", db.get_table_names())


Custom DB tables: ['AWBuildVersion', 'DatabaseLog', 'ErrorLog']


In [131]:
from sqlalchemy import inspect
insp = inspect(engine)
print("dbo tables:", insp.get_table_names(schema="dbo"))
print("Person tables:", insp.get_table_names(schema="Person"))


dbo tables: ['AWBuildVersion', 'DatabaseLog', 'ErrorLog']
Person tables: ['Address', 'AddressType', 'BusinessEntity', 'BusinessEntityAddress', 'BusinessEntityContact', 'ContactType', 'CountryRegion', 'EmailAddress', 'Password', 'Person', 'PersonPhone', 'PhoneNumberType', 'StateProvince']


In [132]:
def _get_all_tables(self):
    from sqlalchemy import inspect
    return set(inspect(self.engine).get_table_names())


In [135]:
from langchain.sql_database import SQLDatabase

class AllSchemasSQLDatabase(SQLDatabase):
    def _get_all_tables(self):
        # Bypass the engine inspector completely by using our pre-populated metadata.
        return set(self.metadata.tables.keys())

# Use your fully reflected metadata.
db = AllSchemasSQLDatabase(engine=engine, metadata=metadata)
print("Custom DB tables:", db._get_all_tables())


AttributeError: 'AllSchemasSQLDatabase' object has no attribute 'metadata'

In [134]:
print("Custom DB tables:", db._get_all_tables())

AttributeError: 'AllSchemasSQLDatabase' object has no attribute 'metadata'

In [136]:
from langchain.sql_database import SQLDatabase

class AllSchemasSQLDatabase(SQLDatabase):
    def _get_all_tables(self):
        # Directly use the metadata's table keys.
        return set(self.metadata.tables.keys())

# Use the custom subclass.
db = AllSchemasSQLDatabase(engine=engine, metadata=metadata)
print("Custom DB tables:", db.get_table_names())


Custom DB tables: ['AWBuildVersion', 'DatabaseLog', 'ErrorLog']


In [137]:
from sqlalchemy import MetaData

metadata_dbo = MetaData()
metadata_person = MetaData()
metadata_sales = MetaData()

metadata_dbo.reflect(bind=engine, schema="dbo")
metadata_person.reflect(bind=engine, schema="Person")
metadata_sales.reflect(bind=engine, schema="Sales")

# Merge tables into a new metadata object.
merged_metadata = MetaData()
for m in [metadata_dbo, metadata_person, metadata_sales]:
    for table in m.tables.values():
        table.tometadata(merged_metadata)

print("Merged tables:", list(merged_metadata.tables.keys()))


C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\3125609081.py:8: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  metadata_person.reflect(bind=engine, schema="Person")
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\3125609081.py:9: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  metadata_sales.reflect(bind=engine, schema="Sales")
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\3125609081.py:9: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  metadata_sales.reflect(bind=engine, schema="Sales")


Merged tables: ['dbo.AWBuildVersion', 'dbo.DatabaseLog', 'dbo.ErrorLog', 'Person.Address', 'Person.StateProvince', 'Person.CountryRegion', 'Sales.SalesTerritory', 'Person.AddressType', 'Person.BusinessEntity', 'Person.BusinessEntityAddress', 'Person.BusinessEntityContact', 'Person.ContactType', 'Person.Person', 'Person.EmailAddress', 'Person.Password', 'Person.PersonPhone', 'Person.PhoneNumberType', 'Sales.CountryRegionCurrency', 'Sales.Currency', 'Sales.CreditCard', 'Sales.CurrencyRate', 'Sales.Customer', 'Sales.Store', 'Sales.SalesPerson', 'HumanResources.Employee', 'Sales.PersonCreditCard', 'Sales.SalesOrderDetail', 'Sales.SalesOrderHeader', 'Purchasing.ShipMethod', 'Sales.SpecialOfferProduct', 'Production.Product', 'Production.ProductModel', 'Production.ProductSubcategory', 'Production.ProductCategory', 'Production.UnitMeasure', 'Sales.SpecialOffer', 'Sales.SalesOrderHeaderSalesReason', 'Sales.SalesReason', 'Sales.SalesPersonQuotaHistory', 'Sales.SalesTaxRate', 'Sales.SalesTerritor

C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\3125609081.py:15: SADeprecationWarning: Table.tometadata() is renamed to Table.to_metadata() (deprecated since: 1.4)
  table.tometadata(merged_metadata)
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\3125609081.py:15: SAWarning: Table 'CountryRegion' already exists within the given MetaData - not copying.
  table.tometadata(merged_metadata)
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\3125609081.py:15: SAWarning: Table 'Person' already exists within the given MetaData - not copying.
  table.tometadata(merged_metadata)
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\3125609081.py:15: SAWarning: Table 'BusinessEntity' already exists within the given MetaData - not copying.
  table.tometadata(merged_metadata)
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_31396\3125609081.py:15: SAWarning: Table 'SalesTerritory' already exists within the given MetaData - not copying.
  table.tometadata(merged_metadata)
C:\Users\Sreeha

In [138]:
db = AllSchemasSQLDatabase(engine=engine, metadata=merged_metadata)
print("Merged DB tables:", db.get_table_names())


Merged DB tables: ['AWBuildVersion', 'DatabaseLog', 'ErrorLog']
